# hsimple
 This program creates :
   - a one dimensional histogram
   - a two dimensional histogram
   - a profile histogram
   - a memory-resident ntuple

 These objects are filled with some random numbers and saved on a file.




**Author:** Wim Lavrijsen, Enric Tejedor  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Tuesday, March 19, 2024 at 07:03 PM.</small></i>

In [1]:
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F
from ROOT import gROOT, gBenchmark, gRandom, gSystem
import ctypes

Welcome to JupyROOT 6.26/06


Create a new canvas, and customize it.

In [2]:
c1 = TCanvas( 'c1', 'Dynamic Filling Example', 200, 10, 700, 500 )
c1.SetFillColor( 42 )
c1.GetFrame().SetFillColor( 21 )
c1.GetFrame().SetBorderSize( 6 )
c1.GetFrame().SetBorderMode( -1 )

Create a new ROOT binary machine independent file.
Note that this file may contain any kind of ROOT objects, histograms,
pictures, graphics objects, detector geometries, tracks, events, etc..
This file is now becoming the current directory.

In [3]:
hfile = gROOT.FindObject( 'py-hsimple.root' )
if hfile:
   hfile.Close()
hfile = TFile( 'py-hsimple.root', 'RECREATE', 'Demo ROOT file with histograms' )

Create some histograms, a profile histogram and an ntuple

In [4]:
hpx    = TH1F( 'hpx', 'This is the px distribution', 100, -4, 4 )
hpxpy  = TH2F( 'hpxpy', 'py vs px', 40, -4, 4, 40, -4, 4 )
hprof  = TProfile( 'hprof', 'Profile of pz versus px', 100, -4, 4, 0, 20 )
ntuple = TNtuple( 'ntuple', 'Demo ntuple', 'px:py:pz:random:i' )

Set canvas/frame attributes.

In [5]:
hpx.SetFillColor( 48 )

gBenchmark.Start( 'hsimple' )

Initialize random number generator.

In [6]:
gRandom.SetSeed()
rannor, rndm = gRandom.Rannor, gRandom.Rndm

For speed, bind and cache the Fill member functions,

In [7]:
histos = [ 'hpx', 'hpxpy', 'hprof', 'ntuple' ]
for name in histos:
   exec('%sFill = %s.Fill' % (name,name))

Fill histograms randomly.

In [8]:
px_ref, py_ref = ctypes.c_double(), ctypes.c_double()
kUPDATE = 1000
for i in range( 25000 ):
 # Generate random values. Use ctypes to pass doubles by reference
   rannor( px_ref, py_ref )
 # Retrieve the generated values
   px = px_ref.value
   py = py_ref.value
   
   pz = px*px + py*py
   random = rndm(1)

 # Fill histograms.
   hpx.Fill( px )
   hpxpy.Fill( px, py )
   hprof.Fill( px, pz )
   ntuple.Fill( px, py, pz, random, i )

 # Update display every kUPDATE events.
   if i and i%kUPDATE == 0:
      if i == kUPDATE:
         hpx.Draw()

      c1.Modified()
      c1.Update()

      if gSystem.ProcessEvents():            # allow user interrupt
         break

Destroy member functions cache.

In [9]:
for name in histos:
   exec('del %sFill' % name)
del histos

gBenchmark.Show( 'hsimple' )

hsimple   : Real Time =   7.40 seconds Cpu Time =   0.21 seconds


Save all objects in this file.

In [10]:
hpx.SetFillColor( 0 )
hfile.Write()
hpx.SetFillColor( 48 )
c1.Modified()
c1.Update()

Note that the file is automatically closed when application terminates
or when the file destructor is called.

Draw all canvases 

In [11]:
%jsroot on
from ROOT import gROOT 
gROOT.GetListOfCanvases().Draw()